# Machine learning tutorial: R edition

I developed this tutorial for a presentation I was giving to the University of Guelph Integrative Biology R users group. The topic was an introduction to the implementation of machine learning algorithms in R.

### Who can benefit from this?

This tutorial is a good first step for someone looking to learn the steps needed for exploring data, cleaning data, and training/evaluating some basic machine learning algorithms. It is also a useful resource for someone who is comfortable doing data science in other languages such as python and wants to learn how to apply their data science skills in R. As a fun exercise you could compare this code to the python code in the book listed below.

Data and code come from chapter 2 of this book: https://github.com/ageron/handson-ml

Here I have 'translated' (and heavily abridged) the code from python into R so that it can be used as a good intro example for how to implement some machine learning algorithms. The workflow isn't exactly the same as the book but the data arrives cleaned at roughly the same point. 

I've chosen this dataset because:
1. It is freely avaliable online so we won't get sued.
2. It is 'medium' sized. Not small enough to feel overly toyish, but not so big as to be cumbersome.
3. There are a reasonable number of predictor columns, so it isn't too much to take in and understand what they all mean.

The columns are as follows, their names are pretty self explanitory:

longitude

latitude

housing_median_age

total_rooms

total_bedrooms

population

households

median_income

median_house_value

ocean_proximity

Each row pertains to a group of houses (I forget if this is by block or postal code but the important bit is they are medians because it is a bunch of houses in close proximity grouped together).

## Step 1. Load in the data.

If you missed the email link, download 'housing.csv' from here:

https://github.com/ageron/handson-ml/tree/master/datasets/housing

Then adjust the following code to your directory of choice.

In [ ]:
library(tidyverse)
library(reshape2)

In [ ]:
housing = read.csv('../housing.csv')

First thing I always do is use the head command to make sure the data isn't weird and looks how I expected.

In [ ]:
head(housing)

Next I always call summary, just to see if the #s are #s and the categoricals are categoricals.

In [ ]:
summary(housing)

So from that summary we can see a few things we need to do before actually running algorithms.
1. NA's in total_bedrooms need to be addressed. These must be given a value
2. We will split the ocean_proximity into binary columns. Most machine learning algorithms in R can handle categoricals in a single column, but we will cater to the lowest common denominator and do the splitting.
3. Make the total_bedrooms and total_rooms into a mean_number_bedrooms and mean_number_rooms columns as there are likely more accurate depections of the houses in a given group.


In [ ]:
par(mfrow=c(2,5))

In [ ]:
colnames(housing)

Lets take a gander at the variables

In [ ]:
ggplot(data = melt(housing), mapping = aes(x = value)) + 
    geom_histogram(bins = 30) + facet_wrap(~variable, scales = 'free_x')

Things I see from this:
1. There are some housing blocks with old age homes in them.
2. The median house value has some weird cap applied to it causing there to be a blip at the rightmost point on the hist. There are most definitely houses in the bay area worth more than 500,000... even in the 90s when this data was collected!
3. We should standardize the scale of the data for any non-tree based methods. As some of the variables range from 0-10, while others go up to 500,000
4. We need to think about how the cap on housing prices can affect our prediction... may be worth removing the capped values and only working with the data we are confident in.

## Step 2. Clean the data

### Impute missing values
Fill median for total_bedrooms which is the only column with missing values. The median is used instead of mean because it is less influenced by extreme outliers. Note this may not be the best, as these could be actual buildings with no bedrooms (warehouses or something). We don't know... but imputation is often the best of a bad job

In [ ]:
housing$total_bedrooms[is.na(housing$total_bedrooms)] = median(housing$total_bedrooms , na.rm = TRUE)

### Fix the total columns - make them means

In [ ]:
housing$mean_bedrooms = housing$total_bedrooms/housing$households
housing$mean_rooms = housing$total_rooms/housing$households

drops = c('total_bedrooms', 'total_rooms')

housing = housing[ , !(names(housing) %in% drops)]

In [ ]:
head(housing)

### Turn categoricals into booleans

Below I do the following:
1. Get a list of all the categories in the 'ocean_proximity' column
2. Make a new empty dataframe of all 0s, where each category is its own colum
3. Use a for loop to populate the appropriate columns of the dataframe
4. Drop the original column from the dataframe.

This is an example of me coding R with a python accent... I would love comments about how to do this more cleanly in R!

Fun follow up task: can you turn this into a function that could be used to split any categorial column?

In [ ]:
categories = unique(housing$ocean_proximity)
#split the categories off
cat_housing = data.frame(ocean_proximity = housing$ocean_proximity)

In [ ]:
for(cat in categories){
    cat_housing[,cat] = rep(0, times= nrow(cat_housing))
}
head(cat_housing) #see the new columns on the right

In [ ]:
for(i in 1:length(cat_housing$ocean_proximity)){
    cat = as.character(cat_housing$ocean_proximity[i])
    cat_housing[,cat][i] = 1
}

head(cat_housing)

In [ ]:
cat_columns = names(cat_housing)
keep_columns = cat_columns[cat_columns != 'ocean_proximity']
cat_housing = select(cat_housing,one_of(keep_columns))

tail(cat_housing)

## Scale the numerical variables

Note here I scale every one of the numericals except for 'median_house_value' as this is what we will be working to predict. The x values are scaled so that coefficients in things like support vector machines are given equal weight, but the y value scale doen't affect the learning algorithms in the same way (and we would just need to re-scale the predictions at the end which is another hassle).

In [ ]:
colnames(housing)

In [ ]:
drops = c('ocean_proximity','median_house_value')
housing_num =  housing[ , !(names(housing) %in% drops)]

In [ ]:
head(housing_num)

In [ ]:
scaled_housing_num = scale(housing_num)

In [ ]:
head(scaled_housing_num)

## Merge the altered numerical and categorical dataframes

In [ ]:
cleaned_housing = cbind(cat_housing, scaled_housing_num, median_house_value=housing$median_house_value)

In [ ]:
head(cleaned_housing)

## Step 3. Create a test set of data
We pull this subsection from the main dataframe and put it to the side to not be looked at prior to testing our models. Don't look at it, as snooping the test data introduces a bias to your work!

This is the data we use to validate our model, when we train a machine learning algorithm the goal is usually to make an algorithm that predicts well on data it hasn't seen before. To assess this feature, we pull a set of data to validate the models as accurate/inaccurate once we have completed the training process.

In [ ]:
set.seed(1738) # Set a random seed so that same sample can be reproduced in future runs

sample = sample.int(n = nrow(cleaned_housing), size = floor(.8*nrow(cleaned_housing)), replace = F)
train = cleaned_housing[sample, ] #just the samples
test  = cleaned_housing[-sample, ] #everything but the samples

I like to use little sanity checks like the ones below to make sure the manipulations have done what I want.
With big dataframes you need find ways to be sure that don't involve looking at the whole thing every step!

Note that the train data below has all the columns we want, and also that the index is jumbled (so we did take a random sample). The second check makes sure that the length of the train and test dataframes equals the length of the dataframe they were split from, which shows we didn't lose data or make any up by accident!

In [ ]:
head(train)

In [ ]:
nrow(train) + nrow(test) == nrow(cleaned_housing)

## Step 4. Test some predictive models.

We start here with just a simple linear model using 3 of the avaliable predictors. Median income, total rooms and population. This serves as an entry point to introduce the topic of cross validation and a basic model. We want a model that makes good predictions on data that it has not seen before. A model that explains the variation in the data it was trained on well, but does not generalize to external data is referred to as being overfit. You may thin "that's why we split off some test data!" but we don't want to repeatedly assess against our test set, as then the model can just become overfit to that set of data thus moving and not solving the problem. 

So here we do cross validation to test the model using the training data itself. Our K is 5, what this means is that the training data is split into 5 equal portions. One of the 5 folds is put to the side (as a mini test data set) and then the model is trained using the other 4 portions. After that the predictions are made on the folds that was withheld, and the process is repeated for each of the 5 folds and the average predictions produced from the iterations of the model is taken. This gives us a rough understanding of how well the model predicts on external data!

In [ ]:
library('boot')

In [ ]:
?cv.glm # note the K option for K fold cross validation

In [ ]:
glm_house = glm(median_house_value~median_income+mean_rooms+population, data=cleaned_housing)
k_fold_cv_error = cv.glm(cleaned_housing , glm_house, K=5)

In [ ]:
k_fold_cv_error$delta

The first component is the raw cross-validation estimate of prediction error. 
The second component is the adjusted cross-validation estimate.

In [ ]:
glm_cv_rmse = sqrt(k_fold_cv_error$delta)[1]
glm_cv_rmse #off by about $83,000... it is a start

In [ ]:
names(glm_house) #what parts of the model are callable?

In [ ]:
glm_house$coefficients 

Since we scaled the imputs we can say that of the three we looked at, median income had the biggest effect on housing price... but I'm always very careful and google lots before intrepreting coefficients!

### Random forest model

In [ ]:
library('randomForest')

In [ ]:
?randomForest

In [ ]:
names(train)

In [ ]:
set.seed(1738)

train_y = train[,'median_house_value']
train_x = train[, names(train) !='median_house_value']

head(train_y)
head(train_x)

In [ ]:
#some people like weird r format like this... I find it causes headaches
#rf_model = randomForest(median_house_value~. , data = train, ntree =500, importance = TRUE)
rf_model = randomForest(train_x, y = train_y , ntree = 500, importance = TRUE)

In [ ]:
names(rf_model) #these are all the different things you can call from the model.

In [ ]:
rf_model$importance

Percentage included mean squared error is a measure of feature importance. It is defined as the measure of the increase in mean squared error of predictions when the given variable is shuffled, thereby acting as a metric of that given variable’s importance in the performance of the model. So higher number  ==  more important predictor.

### The out-of-bag (oob) error estimate
In random forests, there is no need for cross-validation or a separate test set to get an unbiased estimate of the test set error. It is estimated internally, during the run, as follows:

Each tree is constructed using a different bootstrap sample from the original data. About one-third of the cases are left out of the bootstrap sample and not used in the construction of the kth tree.

In [ ]:
oob_prediction = predict(rf_model) #leaving out a data source forces OOB predictions

In [ ]:
#you may have noticed that this is avaliable using the $mse in the model options.
#but this way we learn stuff!
train_mse = mean(as.numeric((oob_prediction - train_y)^2))
oob_rmse = sqrt(train_mse)
oob_rmse

So even using a random forest of only 1000 decision trees we are able to predict the median price of a house in a given district to within $49,000 of the actual median house price. This can serve as our bechmark moving forward and trying other models.

How well does the model predict on the test data?

In [ ]:
test_y = test[,'median_house_value']
test_x = test[, names(test) !='median_house_value']


y_pred = predict(rf_model , test_x)
test_mse = mean(((y_pred - test_y)^2))
test_rmse = sqrt(test_mse)
test_rmse

Well that looks great! Our model scored roughly the same on the training and testing data, suggesting that it is not overfit and that it makes good predictions.

## Step 5. Next Steps

So above we have covered the basics of cleaning data and getting a machine learning algorithm up and running in R. But I've on purpose left some room for improvement.

The obvious way to improve the model is to provide it with better data. Recall our columns:

longitude
latitude
housing_median_age
total_rooms
total_bedrooms
population
households
median_income
median_house_value
ocean_proximity

### Suggestions on ways to improve the results
Why not use your R skills to build new data! One suggestion would be to take the longitude and latitude and work with these data. You could try to find things like 'distance to closest city with 1 million people' or other location based stats. This is called feature engineering and data scientists get paid big bucks to do it effectively!

You may also wish to branch out and try some other models to see if they improve over the random forest benchmark we have set. Note this is not an exhaustive list but a starting point

Tree based methods:

gradient boosting - library(gbm)
extreme gradient boosting - library(xgb)

Other fun methods:
support vevtor machines - library(e1071)
neural networks - library(neuralnet)

### Hyperparameters and Grid search

When tuning models the next thing to worry about is the hyperparameters. All this means is the different options you pass into a model when you initialze it. i.e. the hyperparameter in out random forest model was n_tree = x, we chose x = 500, but we could have tried x = 2500, x = 1500, x = 100000 etc. 

Grid search is a common method to find the best combination of hyperparameters (as there are often more than the 1 we see in the random forest example!). Essentially this is where you make every combination of a set of paramaters and run a cross validation on each set, seeing which set gives the best predictions. An alternative is random search. When the number of hyperparameters is high then the computational load of a full grid search may be too much, so a random search takes a subset of the combinations and finds the best one in the random sample (sounds like a crapshoot but it actually works well!). These methods can be implemented easily using a for loop or two... there are also packages avaliable to help with these tasks.

Here we exit the scope of what I can cover in a short tutorial, look at the r package 'caret' it has great functions for streamling things like grid searches for the best parameters. http://caret.r-forge.r-project.org/


## Have you made a sweet model that predicts well or taught you something?
If so, you can submit the script to kaggle here: 

You can post a script or your own kernel (or fork this document and make a better version) up for the world to enjoy! I promise to upvote you if you do.

### Making your own models? Go forth with the train and test dataframes in hand to make your machine learn something!
I have also followed up on this kernel with a few sequels. Take a look at some of the other kerels produced using this dataset! [Here I expand the model through the use of gradient boosting algorithms (also in r)](https://www.kaggle.com/camnugent/gradient-boosting-and-parameter-tuning-in-r) and [here I engineer some new features and increase the prediction accuracy even more (I did this one in python).](https://www.kaggle.com/camnugent/geospatial-feature-engineering-and-visualization)

Also the code from this notebook has been refactored and made more R-like [here!](https://www.kaggle.com/karlcottenie/introduction-to-machine-learning-in-r-tutorial)